# Deep : Labelisation de sentiment des cleints selon leurs avis ! 

In [2]:
import pyodbc
import pandas as pd
from transformers import pipeline

#  Connexion SQL Server

  
conn = pyodbc.connect(
    'DRIVER={SQL Server};SERVER=localhost;DATABASE=SuplyChain_DataWarehouse;Trusted_Connection=yes;'

    'DATABASE=TON_DATAWAREHOUSE;'
    'Trusted_Connection=yes;'
)

# 1. Récupérer les avis
query = "SELECT Avis_PK, avis FROM Dim_Avis;"
df = pd.read_sql(query, conn)

#  2. Charger le modèle de sentiment
classifier = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment",
    device=-1
)

#  3. Mapper les labels vers des sentiments FR
def map_stars_to_sentiment(label):
    if '1' in label or '2' in label:
        return 'NEGATIVE'
    elif '3' in label:
        return 'NEUTRAL'
    else:
        return 'POSITIVE'

#  4. Appliquer l’analyse de sentiment
df['stars'] = df['avis'].apply(lambda x: classifier(x[:512])[0]['label'])
df['sentiment'] = df['stars'].apply(map_stars_to_sentiment)

#  5. Créer la table SQL si elle n'existe pas
cursor = conn.cursor()
cursor.execute("""
IF OBJECT_ID('Dim_Sentiment', 'U') IS NULL
CREATE TABLE Dim_Sentiment (
    Avis_PK INT PRIMARY KEY,
    sentiment VARCHAR(20)
);
""")

#  6. Insérer ou mettre à jour les sentiments
for index, row in df.iterrows():
    cursor.execute("""
        MERGE Dim_Sentiment AS target
        USING (SELECT ? AS Avis_PK, ? AS sentiment) AS source
        ON target.Avis_PK = source.Avis_PK
        WHEN MATCHED THEN
            UPDATE SET sentiment = source.sentiment
        WHEN NOT MATCHED THEN
            INSERT (Avis_PK, sentiment) VALUES (source.Avis_PK, source.sentiment);
    """, row.Avis_PK, row.sentiment)

conn.commit()
cursor.close()
conn.close()

print(" Sentiments insérés dans Dim_Sentiment avec succès.")


C:\Users\user\AppData\Local\Temp\ipykernel_25076\3783323720.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Device set to use cpu


 Sentiments insérés dans Dim_Sentiment avec succès.


# Sentiment : faché happy !! 

In [3]:
import pyodbc
import pandas as pd
from transformers import pipeline

#  Connexion à SQL Server
conn = pyodbc.connect(
    'DRIVER={SQL Server};SERVER=localhost;DATABASE=SuplyChain_DataWarehouse;Trusted_Connection=yes;'

    'DATABASE=TON_DATAWAREHOUSE;'
    'Trusted_Connection=yes;'
)

#  1. Récupérer les avis
query = "SELECT Avis_PK, avis FROM Dim_Avis;"
df = pd.read_sql(query, conn)

#  2. Charger le modèle d'émotions
classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    device=-1
)

#  3. Appliquer la prédiction d'émotion
df['emotion'] = df['avis'].apply(lambda text: classifier(text[:512])[0]['label'])

#  4. Créer la table Dim_Emotion si elle n'existe pas
cursor = conn.cursor()
cursor.execute("""
IF OBJECT_ID('Dim_Emotion', 'U') IS NULL
CREATE TABLE Dim_Emotion (
    Avis_PK INT PRIMARY KEY,
    emotion VARCHAR(50)
);
""")

#  5. Insérer ou mettre à jour les émotions dans SQL
for index, row in df.iterrows():
    cursor.execute("""
        MERGE Dim_Emotion AS target
        USING (SELECT ? AS Avis_PK, ? AS emotion) AS source
        ON target.Avis_PK = source.Avis_PK
        WHEN MATCHED THEN
            UPDATE SET emotion = source.emotion
        WHEN NOT MATCHED THEN
            INSERT (Avis_PK, emotion) VALUES (source.Avis_PK, source.emotion);
    """, row.Avis_PK, row.emotion)

conn.commit()
cursor.close()
conn.close()

print(" Émotions insérées avec succès dans Dim_Emotion.")


C:\Users\user\AppData\Local\Temp\ipykernel_3076\1356505004.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
Device set to use cpu


 Émotions insérées avec succès dans Dim_Emotion.
